In [ ]:
!pip install kaggle

# Download dataset dari kaggle

!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d fedesoriano/stroke-prediction-dataset

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
mkdir: cannot create directory ‘/root/.kaggle’: File exists
  0% 0.00/67.4k [00:00<?, ?B/s]
100% 67.4k/67.4k [00:00<00:00, 18.9MB/s]


In [ ]:
!unzip stroke-prediction-dataset.zip

Archive:  stroke-prediction-dataset.zip
  inflating: healthcare-dataset-stroke-data.csv  


In [ ]:
import os
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn import metrics

In [ ]:
# Set data path

DATA_PATH = '/content/healthcare-dataset-stroke-data.csv'

In [ ]:
# One-hot encoding

data = pd.read_csv(DATA_PATH)
data = data.dropna()
data = data[data["smoking_status"].str.contains("Unknown") == False]
data['gender'].replace(['Female','Male','Other'], [0,1,2],inplace=True)
data['ever_married'].replace(['No','Yes'],[0,1],inplace=True)
data['work_type'].replace(['Self-employed','Private', 'Govt_job', 'children', 'Never_worked'],[0,1,2,3,4],inplace=True)
data['Residence_type'].replace(['Urban','Rural'],[0,1],inplace=True)
data['smoking_status'].replace(['never smoked','formerly smoked','smokes'],[0,1,2],inplace=True)

In [ ]:
# Splitting features dari target

X, y = data.iloc[:, 1:-1], data.iloc[:, [-1]]
y = y.values
y = y.ravel()

In [ ]:
# Membuat model XGBoost
bst = XGBClassifier(n_estimators=2, max_depth= 30,learning_rate= 1, objective='binary:logistic')

# Memisahkan training dan testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2)

# Fitting model dengan data training
bst.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=30, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=2, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [ ]:
# Score model
print(bst.score(X_test, y_test))

0.9314868804664723


In [ ]:
# Predict semua Y dari X keseluruhan
y_pred = bst.predict(X)

In [ ]:
# Metrics dari hasil y dan y prediksi
print(metrics.classification_report(y, y_pred))

              precision    recall  f1-score   support

           0       0.96      1.00      0.98      3246
           1       0.78      0.26      0.38       180

    accuracy                           0.96      3426
   macro avg       0.87      0.63      0.68      3426
weighted avg       0.95      0.96      0.95      3426

